In [1]:
require('tidyverse')
options(stringsAsFactors=FALSE)

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.1     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
SNP.DIR <- '../data/biomarker/m2'

In [4]:
f.list <- list.files(pattern="*.txt", SNP.DIR)

In [3]:
biomarker <- read.table("../phe_extraction/list_biomarker.txt", header=TRUE, stringsAsFactors=FALSE)
head(biomarker)
biomarker$Field <- sapply(biomarker$Field, as.character)

Field,name
30620,Alanine aminotransferase
30600,Albumin
30610,Alkaline phosphatase
30630,Apolipoprotein A
30640,Apolipoprotein B
30650,Aspartate aminotransferase


In [4]:
example_trait <- '30620'
example_tab <- read.delim(sprintf("%s/both_%s.txt", SNP.DIR, example_trait), sep=" ")
COLUMN_NAMES <- c(colnames(example_tab), "category")

In [22]:

reformTabs <- function(trait){
       print(trait)
    snp_file <- sprintf("%s/snp_table_%s.txt", SNP.DIR, trait)
    if (!file.exists(snp_file)){
        print(sprintf("Error, run did not complete for trait %s", trait))
        return(NA)
    }
    snp_tab <- read.delim(snp_file, sep=" ")
   
    # load f and m and both files
    f_file <- sprintf("%s/f_spec_%s.txt", SNP.DIR, trait)
    m_file <- sprintf("%s/m_spec_%s.txt", SNP.DIR, trait)
    both_file <- sprintf("%s/both_%s.txt", SNP.DIR, trait)
    
    f_tab2 = NA # initialize
    m_tab2 = NA # initialize
    both_tab2 = NA # initialize

    
    if (file.exists(f_file)){
        f_tab <- read.delim(f_file, sep=" ")
        if (nrow(f_tab)!=0){
         f_tab2 <- left_join(select(data.frame(f_tab), -post), snp_tab, by="SNP")[,COLUMN_NAMES]
        }
    }
    
    if (file.exists(m_file)){
        m_tab <- read.delim(m_file, sep=" ")
        if (nrow(m_tab)!=0){

            m_tab2 <- left_join(select(data.frame(m_tab),-post), snp_tab, by="SNP")[,COLUMN_NAMES]
        }
    }
    
    if (file.exists(both_file)){
        both_tab <- read.delim(both_file, sep=" ")
        both_tab2 <- left_join(data.frame(both_tab), select(snp_tab, SNP, category), by="SNP")[,COLUMN_NAMES]
    }
    
    # put together
    list_tabs <- list("f"=f_tab2, "m"=m_tab2, "b"=both_tab2)
        
    list_tabs_loaded <- list_tabs[!is.na(list_tabs)]

    if (length(list_tabs_loaded)==0){
        return(NA)
    } else if (length(list_tabs_loaded)==1){
        df <- list_tabs_loaded[[1]]


    } else {
         df <- do.call(rbind, list_tabs_loaded)

    }
   df2 <- cbind(df, rep(trait, nrow(df)))
    return(df2)
}


#list_res <- lapply(biomarker$Field, reformTabs)


In [23]:
list_res <- lapply(biomarker$Field, reformTabs)

[1] "30620"
[1] "30600"
[1] "30610"
[1] "30630"
[1] "30640"
[1] "30650"
[1] "30710"
[1] "30680"
[1] "30690"
[1] "30700"
[1] "30720"
[1] "30660"
[1] "30730"
[1] "30740"
[1] "30750"
[1] "30760"
[1] "30770"
[1] "30780"
[1] "30790"
[1] "30800"
[1] "30810"
[1] "30820"
[1] "30830"
[1] "30850"
[1] "30840"
[1] "30860"
[1] "30870"
[1] "30880"
[1] "30670"
[1] "30890"


In [24]:
list_res2<- list_res[!is.na(list_res)]
length(list_res2)


[1] 25

In [27]:
full_df <- do.call(rbind, list_res2)
head(full_df)
colnames(full_df)[ncol(full_df)] <- "trait"

,SNP,CHR,BP,B_f,B_m,SE_f,SE_m,p_m,p_f,p1,p2,p3,p4,gene,category,"rep(trait, nrow(df))"
m,rs145331563,8,145729793,-1.2353400,-1.2306800,0.10272400,0.09692910,6.44686e-37,2.69515e-33,3.483756e-29,1.100450e-04,9.998864e-01,3.604242e-06,GPT,3,30620
b.1,Affx-15837191,19,3752874,-0.0089499,-0.0223548,0.00365519,0.00366267,1.04082e-09,1.43446e-02,9.821041e-05,9.882935e-12,1.631090e-04,9.997387e-01,APBA3,4,30620
b.2,Affx-19716376,22,44332888,0.0930435,0.0926454,0.00489690,0.00489773,1.04555e-79,2.10594e-80,4.033083e-151,6.720372e-81,3.638836e-81,1.000000e+00,PNPLA3,4,30620
b.3,Affx-20090007,2,44066247,0.0487479,0.0241566,0.00732244,0.00734086,9.99596e-04,2.79812e-11,7.716685e-08,3.265266e-06,4.699158e-13,9.999967e-01,ABCG8,4,30620
b.4,Affx-26682790,5,52193237,0.0158860,0.0370083,0.00661660,0.00665250,2.65530e-08,1.63543e-02,8.538231e-04,1.385713e-10,1.101344e-04,9.990360e-01,ITGA1,4,30620
b.5,Affx-28463585,6,31830593,0.0431294,0.0397117,0.01062330,0.01076600,2.25553e-04,4.91248e-05,1.037415e-02,5.744525e-07,3.713973e-07,9.896249e-01,NEU1,4,30620


In [29]:
tab_w_names <- left_join(full_df, biomarker, by=c("trait"="Field"))
head(tab_w_names)
dim(tab_w_names)


SNP,CHR,BP,B_f,B_m,SE_f,SE_m,p_m,p_f,p1,p2,p3,p4,gene,category,trait,name
rs145331563,8,145729793,-1.2353400,-1.2306800,0.10272400,0.09692910,6.44686e-37,2.69515e-33,3.483756e-29,1.100450e-04,9.998864e-01,3.604242e-06,GPT,3,30620,Alanine aminotransferase
Affx-15837191,19,3752874,-0.0089499,-0.0223548,0.00365519,0.00366267,1.04082e-09,1.43446e-02,9.821041e-05,9.882935e-12,1.631090e-04,9.997387e-01,APBA3,4,30620,Alanine aminotransferase
Affx-19716376,22,44332888,0.0930435,0.0926454,0.00489690,0.00489773,1.04555e-79,2.10594e-80,4.033083e-151,6.720372e-81,3.638836e-81,1.000000e+00,PNPLA3,4,30620,Alanine aminotransferase
Affx-20090007,2,44066247,0.0487479,0.0241566,0.00732244,0.00734086,9.99596e-04,2.79812e-11,7.716685e-08,3.265266e-06,4.699158e-13,9.999967e-01,ABCG8,4,30620,Alanine aminotransferase
Affx-26682790,5,52193237,0.0158860,0.0370083,0.00661660,0.00665250,2.65530e-08,1.63543e-02,8.538231e-04,1.385713e-10,1.101344e-04,9.990360e-01,ITGA1,4,30620,Alanine aminotransferase
Affx-28463585,6,31830593,0.0431294,0.0397117,0.01062330,0.01076600,2.25553e-04,4.91248e-05,1.037415e-02,5.744525e-07,3.713973e-07,9.896249e-01,NEU1,4,30620,Alanine aminotransferase


[1] 19143    17

In [31]:
write.table(tab_w_names, file=sprintf("%s/full_snp_tab.txt", SNP.DIR), sep="\t", row.names=FALSE)

In [37]:
require('reshape2')

df_counts <- data.frame(table(tab_w_names[,c("name", "category")]))
df_counts_short <- dcast(df_counts, name ~ category)
head(df_counts_short)
colnames(df_counts_short) <- c("trait", "f_spec", "m_spec", "both")
write.table()

Loading required package: reshape2

Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths

Using Freq as value column: use value.var to override.


name,2,3,4
Alanine aminotransferase,0,1,439
Albumin,0,0,451
Alkaline phosphatase,1,8,1571
Apolipoprotein A,1,2,1088
Apolipoprotein B,0,0,716
Aspartate aminotransferase,2,0,1003


In [33]:
write.table(filter(tab_w_names, category == 2), file=sprintf("%s/all_f_spec_tab.txt", SNP.DIR), sep="\t", row.names=FALSE)
write.table(filter(tab_w_names, category == 3), file=sprintf("%s/all_m_spec_tab.txt", SNP.DIR), sep="\t", row.names=FALSE)
write.table(filter(tab_w_names, category == 4), file=sprintf("%s/all_cat4_tab.txt", SNP.DIR), sep="\t", row.names=FALSE)

In [ ]:
# put all SNPs together w probabilities

# add the trait ID and name

# create a meta-table of counts of all of traits

In [37]:
 trait <- '30620'

    

[1] "30620"


p1,p2,p3,p4,category,SNP
0.9980048,3.844641e-07,3.597754e-07,1.994453e-03,1,rs115991721
0.9999466,1.162119e-08,7.276292e-08,5.326779e-05,1,rs12562034


[1] 0 1


ERROR: Error: `by` can't contain join column `SNP` which is missing from LHS


In [16]:
# add p1...p4 + category to f_tab, m_tab
f_tab_w_more <- left_join(f_tab, snp_tab)
m_tab_w_more <- left_join(m_tab, snp_tab)
#b
# SHIT need to compute posterior


head(f_tab_w_more, 2)
head(m_tab_w_more, 2)

Joining, by = "SNP"
Joining, by = "SNP"


SNP,CHR,BP,B_f,B_m,SE_f,SE_m,p_m,p_f,post,gene,p1,p2,p3,p4,category
Affx-52354612,15,58838103,0.640991,0.453777,0.0849286,0.0819133,3.03517e-08,4.46767e-14,0.9993879,LIPC,3.550410e-07,0.9993879,1.194965e-06,0.0006105524,2
rs150090666,11,14865399,0.527802,0.271883,0.0595380,0.0574659,2.23413e-06,7.74324e-19,0.9997144,PDE3B,9.396419e-12,0.9997144,3.635398e-13,0.0002855505,2


SNP,CHR,BP,B_f,B_m,SE_f,SE_m,p_m,p_f,post,gene,p1,p2,p3,p4,category
rs118204057,8,19811733,-0.739822,-1.064340,0.0895957,0.0835498,3.77036e-37,1.50280e-16,1.0000000,LPL,1.024458e-29,7.638022e-21,1.0000000,1.079524e-17,3
rs61731956,11,47290147,0.461415,0.521687,0.0722075,0.0737037,1.46768e-12,1.66284e-10,0.5359848,NR1H3,1.099584e-05,9.357140e-03,0.5359848,4.546471e-01,3


In [22]:
both_tab_w_more <- left_join(both_tab, select(snp_tab, SNP, category))
comb_tab <- rbind(f_tab_w_more[,colnames(both_tab_w_more)],m_tab_w_more[,colnames(both_tab_w_more)])
comb_tab2 <- rbind(comb_tab, both_tab_w_more)
dim(comb_tab2)
head(comb_tab2)

Joining, by = "SNP"


[1] 1287   15

SNP,CHR,BP,B_f,B_m,SE_f,SE_m,p_m,p_f,p1,p2,p3,p4,gene,category
Affx-52354612,15,58838103,0.640991,0.453777,0.0849286,0.0819133,3.03517e-08,4.46767e-14,3.550410e-07,9.993879e-01,1.194965e-06,6.105524e-04,LIPC,2
rs150090666,11,14865399,0.527802,0.271883,0.0595380,0.0574659,2.23413e-06,7.74324e-19,9.396419e-12,9.997144e-01,3.635398e-13,2.855505e-04,PDE3B,2
rs188308962,9,107620797,-0.540755,-0.395215,0.0469161,0.0487080,4.93934e-16,1.00853e-30,1.809682e-23,9.124556e-01,1.578385e-15,8.754435e-02,ABCA1,2
rs201079485,11,116661656,-0.618141,-0.592974,0.1108420,0.1090520,5.41078e-08,2.45494e-08,6.898847e-02,6.234499e-01,1.686357e-01,1.389259e-01,APOA5,2
rs118204057,8,19811733,-0.739822,-1.064340,0.0895957,0.0835498,3.77036e-37,1.50280e-16,1.024458e-29,7.638022e-21,1.000000e+00,1.079524e-17,LPL,3
rs61731956,11,47290147,0.461415,0.521687,0.0722075,0.0737037,1.46768e-12,1.66284e-10,1.099584e-05,9.357140e-03,5.359848e-01,4.546471e-01,NR1H3,3


In [25]:
comb_tab2$trait <- "30760"
comb_tab3 <- left_join(comb_tab2, biomarker, by=c("trait"="Field"))
head(comb_tab3)

SNP,CHR,BP,B_f,B_m,SE_f,SE_m,p_m,p_f,p1,p2,p3,p4,gene,category,trait,name
Affx-52354612,15,58838103,0.640991,0.453777,0.0849286,0.0819133,3.03517e-08,4.46767e-14,3.550410e-07,9.993879e-01,1.194965e-06,6.105524e-04,LIPC,2,30760,HDL cholesterol
rs150090666,11,14865399,0.527802,0.271883,0.0595380,0.0574659,2.23413e-06,7.74324e-19,9.396419e-12,9.997144e-01,3.635398e-13,2.855505e-04,PDE3B,2,30760,HDL cholesterol
rs188308962,9,107620797,-0.540755,-0.395215,0.0469161,0.0487080,4.93934e-16,1.00853e-30,1.809682e-23,9.124556e-01,1.578385e-15,8.754435e-02,ABCA1,2,30760,HDL cholesterol
rs201079485,11,116661656,-0.618141,-0.592974,0.1108420,0.1090520,5.41078e-08,2.45494e-08,6.898847e-02,6.234499e-01,1.686357e-01,1.389259e-01,APOA5,2,30760,HDL cholesterol
rs118204057,8,19811733,-0.739822,-1.064340,0.0895957,0.0835498,3.77036e-37,1.50280e-16,1.024458e-29,7.638022e-21,1.000000e+00,1.079524e-17,LPL,3,30760,HDL cholesterol
rs61731956,11,47290147,0.461415,0.521687,0.0722075,0.0737037,1.46768e-12,1.66284e-10,1.099584e-05,9.357140e-03,5.359848e-01,4.546471e-01,NR1H3,3,30760,HDL cholesterol


In [9]:
# SNP extract


source('../mixture_model_scripts/model_utils.R')
#source('heritability_utils.R')
source('../mixture_model_scripts/snp_utils.R')
DATA.FOLDER <- "/scratch/PI/mrivas/users/erflynn/sex_div_gwas/data/"

extractSNPcat <- function(snp.df, df.f, df.m, category, trait){
    comp4 <- snp.df[which(snp.df$category==category),c("p1", "p2", "p3", "p4", "SNP")]
    if (length(comp4$SNP) > 0){
            both.snps <- cbind(df.f[df.f$SNP %in% comp4$SNP ,c("SNP", "CHR", "BP", "BETA","SE", "P")], 
             df.m[df.m$SNP %in% comp4$SNP,c("BETA","SE", "P")])
            colnames(both.snps) <- c("SNP", "CHR", "BP", "B_f", "SE_f", "p_f", "B_m", "SE_m", "p_m")
            both.snp.df <- both.snps[,c("SNP", "CHR", "BP", "B_f", "B_m", "SE_f", "SE_m", "p_m","p_f")] 
            both.snp.df <- merge(both.snp.df, comp4, by="SNP")       
    }   
    both.snp.df2 <- annotateSNP(both.snp.df)

    write.table(both.snp.df2, file=sprintf("%s/biomarker/m2/snps%s_%s.txt", DATA.FOLDER, category, trait), row.names=FALSE)


}



In [11]:
trait <- "Testosterone"
snp.df <- read.delim(sprintf("%s/biomarker/m2/snp_table_%s.txt", DATA.FOLDER, trait), header=TRUE, sep=" ")
head(snp.df)
cat.count <- table(snp.df$category)
print(cat.count)
    if ("2" %in% names(cat.count) | "3" %in% names(cat.count) | "4" %in% names(cat.count)){
        print(trait)
        print(cat.count)
        list.prefixes <- c("zerosex", "onesex")
        chrs <- c(1:22)
        list.ds <- lapply(list.prefixes, function(prefix) {
                all.dat <- do.call(rbind, lapply(chrs, function(chr) { getFile(prefix, chr, trait)}));
                colnames(all.dat)[1:3] <- c("CHR", "BP", "SNP");
                return(all.dat)
            })

        list.ds2 <- extractOverlappingRows(list.ds)
	df.f <- list.ds2[[1]]
	df.m <- list.ds2[[2]]
    print("Extracting")
	extractSNPcat(snp.df, df.f, df.m, 2, trait)
	extractSNPcat(snp.df, df.f, df.m, 3, trait)
	extractSNPcat(snp.df, df.f, df.m, 4, trait)
    } else {
        print(sprintf("No sex-specific SNPs for %s", trait))
    }


p1,p2,p3,p4,category,SNP
0.9961476,3.625712e-04,0.0034798778,9.961560e-06,1,rs115991721
0.9994809,4.207844e-05,0.0004769808,7.369874e-08,1,rs12562034
0.9995419,2.886834e-05,0.0004291996,4.540599e-08,1,rs4040617
0.9987935,1.110715e-04,0.0010949284,4.644337e-07,1,rs79373928
0.9954687,7.214883e-04,0.0037743484,3.544848e-05,1,rs59771807
0.9959427,1.014418e-04,0.0039540320,1.854871e-06,1,rs116452738



     1      2      3      4 
328085    100    480     29 
[1] "Testosterone"

     1      2      3      4 
328085    100    480     29 
[1] 2
[1] 329035     11
[1] "Extracting"
